## Install typescript: tslab 

In [2]:
import { initializeWasm } from '@docknetwork/crypto-wasm-ts'
await initializeWasm();

In [3]:
const EID_CREDENTIAL = {
    name: "Michael Jackson",
    age: "50",
    profession: "RSE",
    project: "EID"
}
const CREDENTIAL_DATA = Object.values(EID_CREDENTIAL)

In [4]:
const stringToBytes = (str: string) => Uint8Array.from(Buffer.from(str, "utf-8"));

In [5]:
import { BBSPlusSignatureParamsG1, BBSPlusKeypairG2 } from '@docknetwork/crypto-wasm-ts';
const messageCount = CREDENTIAL_DATA.length;

const label = stringToBytes("My signature params");

// Deterministically generated params
const paramsDeterministc = BBSPlusSignatureParamsG1.generate(messageCount, label);

// Generating a keypair once signature parameters are created.

const keypair = BBSPlusKeypairG2.generate(paramsDeterministc);
const sk = keypair.secretKey;
const pk = keypair.publicKey;


In [8]:
import { BBSPlusSignatureG1 } from '@docknetwork/crypto-wasm-ts'
// Encoding messages
const encodedMessages = CREDENTIAL_DATA.map(element => stringToBytes(element));

// The signing function will encode bytes to a field element as true is passed
const sig = BBSPlusSignatureG1.generate(encodedMessages, sk, paramsDeterministc, true);

// As the messages are not encoded, pass true to the verification function to make it encode messages before verifying the signature.
sig.verify(encodedMessages, pk, paramsDeterministc, true);

{ verified: true, error: undefined }


# Selective Disclosure 

In [18]:
import {Statement, Statements, MetaStatements, ProofSpec, Witness, Witnesses, CompositeProof} from '@docknetwork/crypto-wasm-ts';

// Prover prepares the attributes he wants to disclose, i.e. attribute index 2 and 4 (indexing is 0-based), and the ones he wants to hide. 
const revealedMsgIndices: Set<number> = new Set([2, 1]); 

// revealedMsgs are the attributes disclosed to the verifier
const revealedMsgs: Map<number, Uint8Array> = new Map([
  [1, encodedMessages[1]],
  [2, encodedMessages[2]]
]); 

const unrevealedMsgs: Map<number, Uint8Array> = new Map([
  [0, encodedMessages[0]],
  [3, encodedMessages[3]]
]); 

// Create a BBS signature, true indicates that attributes/messages are arbitrary bytes and should be encoded first
const statement1 = Statement.bbsPlusSignatureProverConstantTime(paramsDeterministc, revealedMsgs, true);
const statements = new Statements();
statements.add(statement1);

const ms = new MetaStatements();
const proofSpec = new ProofSpec(statements, ms, [], stringToBytes("Awesome Proof - test 001"));

const witness1 = Witness.bbsPlusSignatureConstantTime(sig, unrevealedMsgs, true);
const witnesses = new Witnesses();
witnesses.add(witness1);

const nonce = stringToBytes('a unique nonce given by verifier');
const proof = CompositeProof.generate(proofSpec, witnesses, nonce);



In [13]:
const verifierStatement1 = Statement.bbsPlusSignatureVerifierConstantTime(paramsDeterministc, pk, revealedMsgs, true);
const verifierStatements = new Statements();
verifierStatements.add(verifierStatement1);

const verifierMS = new MetaStatements();
const verifierProofSpec = new ProofSpec(verifierStatements, verifierMS, [], stringToBytes("Awesome Proof - test 001"));

const verifierWitness1 = Witness.bbsPlusSignature(sig, unrevealedMsgs, true);
const verifierWitnesses = new Witnesses();
verifierWitnesses.add(verifierWitness1);

const verifierNonce = stringToBytes('a unique nonce given by verifier');

proof.verify(verifierProofSpec, verifierNonce)

{ verified: true, error: undefined }


In [ ]:
proof.hex